# Plot brightest

This notebook can be used to plot the brightest shots (i.e. shots with scattering signal, if scattering signal is above random noise)


In [ ]:
# Stephan Kuschel, 2019
import matplotlib.pyplot as plt
%matplotlib notebook
import karabo_data
# from tqdm import tqdm_notebook as tqdm # module not found using max-jhub.desy.de python3 and not required for this
import numpy as np
import functools

img_karabo_source = 'SQS_DPU_LIC/CAM/YAG_UPSTR:daqOutput'

def addcolorbar(ax, im, pos='right', size='5%', pad=0.05, orientation='vertical',
                stub=False, max_ticks=None, label=None):
    '''
    add a colorbar to a matplotlib image.
    
    ax -- the axis object the image is drawn in
    im -- the image (return value of ax.imshow(...))
    
    When changed, please update:
    https://gist.github.com/skuschel/85f0645bd6e37509164510290435a85a
    
    Stephan Kuschel, 2018
    '''
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    import matplotlib.pyplot as plt
    divider = make_axes_locatable(ax)
    cax = divider.append_axes(pos, size=size, pad=pad)
    if stub:
        cax.set_visible(False)
        return cax
    
    cb = plt.colorbar(im, cax=cax, orientation=orientation)
    if max_ticks is not None:
        from matplotlib import ticker
        tick_locator = ticker.MaxNLocator(nbins=max_ticks)
        cb.locator = tick_locator
        cb.update_ticks()
    if label is not None:
        cb.set_label(label)
    return cax

In [ ]:
@functools.lru_cache(maxsize=2)
def getbrightest(runn, rawdir='/gpfs/exfel/exp/SQS/201802/p002195/raw/'):
    # delay is the scanned variable
    run = karabo_data.RunDirectory(rawdir+'r{:04n}'.format(runn))
    imges = np.asarray(run.get_array(img_karabo_source, 'data.image.pixels'))
    #delay = np.asarray(run.get_array('SQS_NQS_CRSC/TSYS/PARKER_TRIGGER', 'actualDelay.value'))
    print('shape of image stream: {}'.format(imges.shape))
    imgyield = np.mean(imges, axis=(1,2))
    #print(imgyield.shape)
    brightidx = np.argsort(imgyield)[::-1]
    return brightidx, imges

In [ ]:
def plotbrightest(runn,rawdir='/gpfs/exfel/exp/SQS/201802/p002195/raw/'):
    #title gives the mean value of image
    print("Wait for image processing ...")
    brightidx, images = getbrightest(runn,rawdir=rawdir)
    print("Plotting ...")
    fig, axss = plt.subplots(5,4, sharex=True, sharey=True)
    for i, ax in enumerate(np.asarray(axss).flatten()):
        bild = images[brightidx[i]]
        im = ax.imshow(bild)
        addcolorbar(ax,im)
        ax.set_title('{:.1f}'.format(np.mean(bild)))
    
    fig.set_size_inches(10,12)
    fig.tight_layout()
    fig.suptitle('run {} -- {} shots'.format(runn, len(brightidx)))
    fig.savefig('brightest of run {}.pdf'.format(runn))

# Start editing here

In [ ]:
# just select which run shall be used. the data will automatically be saved to a pdf in the current folder.
# path for raw exp data: /gpfs/exfel/exp/SQS/201802/p002195/raw/
# path for scratch data: /gpfs/exfel/exp/SQS/201802/p002195/scratch/
plotbrightest(100,rawdir='/gpfs/exfel/exp/SQS/201802/p002195/raw/')